In [1]:
# CELL 1: Imports & GPU Check
import os, json, pathlib, warnings
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models, applications, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras import mixed_precision
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.utils.class_weight import compute_class_weight
from glob import glob

warnings.filterwarnings('ignore')
mixed_precision.set_global_policy('mixed_float16')

print("GPUs:", tf.config.list_physical_devices('GPU'))

GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# CELL 2: Configuration
BASE_DATA_PATH = pathlib.Path("/Volumes/Sarbajit/Model Optimization/Data/Augmented")

modalities = {
    # 'Brain_Tumor': 'Brain Tumor',
    # 'Breast_Cancer': 'Breast Cancer',
    # 'Kidney_Cancer': 'Kidney Cancer',
    'Lymphoma': 'Lymphoma',
    'Leukemia': 'Leukemia',
    'Lung_Colon_Cancer': 'Lung and Colon Cancer',
    'Oral_Cancer': 'Oral Cancer',
    'Skin_Leison': 'Skin Leison',
    'Chest_Xray': 'chest_xray',
}

OUTPUT_ROOT = pathlib.Path("/Volumes/Sarbajit/Model Optimization/Transfer Learning Model/Output")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

BATCH_SIZE = 16
EPOCHS = 100
LR = 1e-4
INPUT_SHAPE = (224, 224, 3)
PATIENCE = 3

In [3]:
# ============================================================
# CELL 3: Helper Functions (fixed for shape & label consistency)
# ============================================================

def get_class_names(gen):
    return list(gen.class_indices.keys())

def compute_class_weights(train_gen):
    """Compute class weights from training generator."""
    labels = train_gen.classes
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(labels),
        y=labels
    )
    cw_dict = {i: w for i, w in enumerate(class_weights)}
    # Ensure all classes have weights
    for i in range(len(train_gen.class_indices)):
        cw_dict.setdefault(i, 1.0)
    return cw_dict

def save_history_plot(hist, out_dir):
    df = pd.DataFrame(hist.history)
    for cols, name in [(['accuracy', 'val_accuracy'], 'accuracy'), (['loss', 'val_loss'], 'loss')]:
        plt.figure(figsize=(10, 8))
        df[cols].plot(title=name.capitalize())
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(out_dir / f"{name}_curve.png", dpi=600)
        plt.close()

def save_confusion_matrix(y_true, y_pred, classes, out_dir):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(classes)))
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig(out_dir / 'confusion_matrix.png', dpi=600)
    plt.close()

def save_classification_report(y_true, y_pred, classes, out_dir):
    report = classification_report(y_true, y_pred,
                                   target_names=classes, output_dict=True, zero_division=0)
    json.dump(report, open(out_dir / 'classification_report.json', 'w'), indent=2)
    with open(out_dir / 'classification_report.txt', 'w') as f:
        f.write(classification_report(y_true, y_pred, target_names=classes, zero_division=0))
    df = pd.DataFrame(report).T.iloc[:-3][['precision', 'recall', 'f1-score']]
    df.plot(kind='bar', figsize=(10, 8), title='Per-Class Metrics')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(out_dir / 'classification_metrics.png', dpi=600)
    plt.close()

def save_roc_auc(y_true_onehot, y_pred_proba, classes, out_dir):
    """Robust ROC AUC plotting for binary and multiclass cases."""
    plt.figure(figsize=(10, 8))

    # Handle binary classification
    if len(classes) == 2:
        # Ensure consistent 2D shape
        if y_pred_proba.ndim == 1 or y_pred_proba.shape[1] == 1:
            y_pred_proba = np.hstack([1 - y_pred_proba.reshape(-1, 1),
                                      y_pred_proba.reshape(-1, 1)])

        fpr, tpr, _ = roc_curve(y_true_onehot[:, 1], y_pred_proba[:, 1])
        auc = roc_auc_score(y_true_onehot[:, 1], y_pred_proba[:, 1])
        plt.plot(fpr, tpr, label=f'ROC (AUC = {auc:.3f})')

    # Multiclass case
    else:
        for i, cls in enumerate(classes):
            fpr, tpr, _ = roc_curve(y_true_onehot[:, i], y_pred_proba[:, i])
            auc = roc_auc_score(y_true_onehot[:, i], y_pred_proba[:, i])
            plt.plot(fpr, tpr, label=f'{cls} (AUC = {auc:.3f})')
        # Add macro-average ROC
        macro_auc = roc_auc_score(y_true_onehot, y_pred_proba,
                                  average='macro', multi_class='ovr')
        plt.plot([], [], ' ', label=f'Macro-AUC = {macro_auc:.3f}')

    plt.plot([0, 1], [0, 1], '--', color='gray')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(out_dir / 'roc_auc.png', dpi=600)
    plt.close()

In [4]:
# ============================================================
# CELL 4: Model Builders (unchanged, just stylistic cleanup)
# ============================================================

def create_resnet50v2_model(num_classes):
    base = applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    base.trainable = False
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dropout(0.3)(x)
    if num_classes == 2:
        head = layers.Dense(1, activation='sigmoid', name='head')(x)
        loss = 'binary_crossentropy'
    else:
        head = layers.Dense(num_classes, activation='softmax', name='head')(x)
        loss = 'categorical_crossentropy'
    model = models.Model(inputs=base.input, outputs=head, name=f'ResNet50V2_{num_classes}cls')
    model.compile(optimizer=optimizers.Adam(LR), loss=loss, metrics=['accuracy'])
    return model

def create_vgg19_model(num_classes):
    base = applications.VGG19(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    base.trainable = False
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dropout(0.3)(x)
    if num_classes == 2:
        head = layers.Dense(1, activation='sigmoid')(x)
        loss = 'binary_crossentropy'
    else:
        head = layers.Dense(num_classes, activation='softmax')(x)
        loss = 'categorical_crossentropy'
    model = models.Model(base.input, head, name=f'VGG19_{num_classes}cls')
    model.compile(optimizer=optimizers.Adam(LR), loss=loss, metrics=['accuracy'])
    return model

def create_mobilenetv3_model(num_classes):
    base = applications.MobileNetV3Large(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    base.trainable = False
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dropout(0.3)(x)
    if num_classes == 2:
        head = layers.Dense(1, activation='sigmoid')(x)
        loss = 'binary_crossentropy'
    else:
        head = layers.Dense(num_classes, activation='softmax')(x)
        loss = 'categorical_crossentropy'
    model = models.Model(base.input, head, name=f'MobileNetV3_{num_classes}cls')
    model.compile(optimizer=optimizers.Adam(LR), loss=loss, metrics=['accuracy'])
    return model

MODEL_BUILDERS = {
    'resnet50v2': create_resnet50v2_model,
    'vgg19': create_vgg19_model,
    'mobilenetv3': create_mobilenetv3_model,
}


In [5]:
# ============================================================
# CELL 5: Data Loading & Generators (unchanged except safety)
# ============================================================

def load_dataframe(dataset_folder):
    paths, labels = [], []
    for cls in sorted(os.listdir(dataset_folder)):
        cls_path = dataset_folder / cls
        if not cls_path.is_dir():
            continue
        for ext in ('*.jpg', '*.jpeg', '*.png', '*.bmp'):
            found = glob(str(cls_path / ext))
            paths.extend(found)
            labels.extend([cls] * len(found))
    df = pd.DataFrame({'Filepath': paths, 'Label': labels})
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df

def create_generators(df):
    train_df, temp = train_test_split(df, test_size=0.30, stratify=df['Label'], random_state=42)
    val_df, test_df = train_test_split(temp, test_size=0.50, stratify=temp['Label'], random_state=42)

    n_classes = df['Label'].nunique()
    class_mode = 'binary' if n_classes == 2 else 'categorical'

    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    )

    train_gen = datagen.flow_from_dataframe(
        train_df, x_col='Filepath', y_col='Label',
        target_size=INPUT_SHAPE[:2], batch_size=BATCH_SIZE,
        class_mode=class_mode, shuffle=True
    )
    val_gen = datagen.flow_from_dataframe(
        val_df, x_col='Filepath', y_col='Label',
        target_size=INPUT_SHAPE[:2], batch_size=BATCH_SIZE,
        class_mode=class_mode, shuffle=False
    )
    test_gen = datagen.flow_from_dataframe(
        test_df, x_col='Filepath', y_col='Label',
        target_size=INPUT_SHAPE[:2], batch_size=BATCH_SIZE,
        class_mode=class_mode, shuffle=False
    )
    return train_gen, val_gen, test_gen

In [6]:
# ============================================================
# CELL 6: MAIN SEQUENTIAL TRAINING LOOP (fixed version)
# ============================================================

for short_name, pretty_name in modalities.items():
    dataset_path = BASE_DATA_PATH / pretty_name
    if not dataset_path.exists():
        print(f"[SKIP] {dataset_path} not found")
        continue

    print(f"\n{'='*80}")
    print(f"DATASET: {pretty_name} ({short_name})")
    print(f"{'='*80}")

    df = load_dataframe(dataset_path)
    n_classes = df['Label'].nunique()
    print(f"   → {len(df)} images | {n_classes} classes")

    # Generators (224x224 only)
    train_gen, val_gen, test_gen = create_generators(df)
    class_names = get_class_names(train_gen)
    class_weights = compute_class_weights(train_gen)
    print(f"   → Class weights: {class_weights}")

    for model_key, builder in MODEL_BUILDERS.items():
        print(f"\n--- Training {model_key.upper()} on {short_name} ---")
        out_dir = OUTPUT_ROOT / model_key / short_name
        out_dir.mkdir(parents=True, exist_ok=True)

        # Build model
        model = builder(n_classes)

        # Callbacks
        ckpt_path = out_dir / f"{model_key}_{short_name}_best.h5"
        callbacks_list = [
            callbacks.EarlyStopping(monitor='val_accuracy', patience=PATIENCE,
                                    restore_best_weights=True, verbose=1)
        ]

        # Train
        hist = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=EPOCHS,
            callbacks=callbacks_list,
            class_weight=class_weights,
            verbose=1
        )

        # Save final model
        final_path = out_dir / f"{model_key}_{short_name}_final.h5"
        model.save(final_path)

        # Visualizations
        save_history_plot(hist, out_dir)

        # Test evaluation
        test_gen.reset()
        true_cls = test_gen.classes

        # Binary vs multiclass logic (robust)
        pred_proba = model.predict(test_gen, verbose=1)
        if n_classes == 2:
            if pred_proba.ndim > 1:
                pred_proba = pred_proba.ravel()
            pred_cls = (pred_proba > 0.5).astype(int)
            true_onehot = to_categorical(true_cls, 2)
        else:
            pred_cls = np.argmax(pred_proba, axis=1)
            true_onehot = to_categorical(true_cls, n_classes)

        # Metrics & plots
        save_confusion_matrix(true_cls, pred_cls, class_names, out_dir)
        save_classification_report(true_cls, pred_cls, class_names, out_dir)
        save_roc_auc(true_onehot, pred_proba, class_names, out_dir)

        # Model architecture visualization
        plot_model(model, to_file=out_dir / 'architecture.png', show_shapes=True, dpi=180)

        # Summary JSON
        summary = {
            'dataset': pretty_name,
            'model': model_key,
            'num_classes': n_classes,
            'test_accuracy': float(np.mean(pred_cls == true_cls)),
            'best_model': str(ckpt_path),
            'final_model': str(final_path),
            'class_weights': class_weights
        }
        json.dump(summary, open(out_dir / 'summary.json', 'w'), indent=2)
        print(f"Done → {out_dir}")

print("\nALL TRAINING COMPLETED! Output folder:")
print(OUTPUT_ROOT)


DATASET: Lymphoma (Lymphoma)
   → 4500 images | 3 classes
Found 3150 validated image filenames belonging to 3 classes.
Found 675 validated image filenames belonging to 3 classes.
Found 675 validated image filenames belonging to 3 classes.
   → Class weights: {0: 1.0, 1: 1.0, 2: 1.0}

--- Training RESNET50V2 on Lymphoma ---


2025-11-07 23:51:57.439912: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-11-07 23:51:57.439944: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-11-07 23:51:57.439947: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2025-11-07 23:51:57.439962: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-07 23:51:57.439971: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2025-11-07 23:51:59.146950: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


197/197 ━━━━━━━━━━━━━━━━━━━━ 28s 123ms/step - accuracy: 0.3794 - loss: 1.2294 - val_accuracy: 0.4593 - val_loss: 1.0275
Epoch 2/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 21s 107ms/step - accuracy: 0.4632 - loss: 1.0575 - val_accuracy: 0.5585 - val_loss: 0.9136
Epoch 3/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 22s 110ms/step - accuracy: 0.5124 - loss: 0.9662 - val_accuracy: 0.6341 - val_loss: 0.8487
Epoch 4/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 21s 108ms/step - accuracy: 0.5721 - loss: 0.9071 - val_accuracy: 0.6637 - val_loss: 0.7986
Epoch 5/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 39s 196ms/step - accuracy: 0.5981 - loss: 0.8603 - val_accuracy: 0.6993 - val_loss: 0.7624
Epoch 6/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 52s 263ms/step - accuracy: 0.6286 - loss: 0.8216 - val_accuracy: 0.7081 - val_loss: 0.7326
Epoch 7/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 52s 263ms/step - accuracy: 0.6375 - loss: 0.8013 - val_accuracy: 0.7304 - val_loss: 0.7075
Epoch 8/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 51s 259ms/step - accuracy: 0.6489 - loss: 0.779

...

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>